In [319]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
from collections import Counter

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [320]:
scraping_date = '_26_04_2021'
data = pd.read_csv('./data/full_profiles'+scraping_date+'.csv')

In [321]:
data.head(3)

,name,languages,description,rating,nb_reviews,Price/hour in USD,skills,industry_expertise,nb_projects,nb_buyers,last_active,gender,city,country,score
0,Maria H.,['English'],We are a small UK based company speciaIising i...,4.9,4694,34.74,"['animation', 'graphics design', 'brand design...","['IT', 'Internet', 'Marketing', 'Media', 'Tech...",7672,4168,26-04-2021,female,Sheffield,United Kingdom,10.043
1,Cormac Reynolds,['English'],Looking for a link building solution or some g...,5.0,1190,138.95,"['online marketing', 'link building', 'marketi...",[],1085,533,25-04-2021,unknown,City of London,United Kingdom,8.691
2,Denise Toepel,"['English', 'Spanish', 'French']",I am a Translation Specialist. I work in publ...,0.0,0,27.79,"['english translation', 'french english', 'on...","['Education', 'Telecommunications']",0,0,25-04-2021,female,Cuenca,Ecuador,0.000


# Remove inactive profiles (no projects) and drop unnucessary features

In [322]:
print('Number of workers with inactive profiles:',  len(data.loc[(data['nb_projects'] == 0)]))

Number of workers with inactive profiles: 2930


In [323]:
data.drop(data.loc[data['nb_projects']==0].index, inplace=True)

Almost 30% of all the profiles are thus deleted of our regression analysis. This represents a big proportion of the profiles that we collected but there is still a significant amount of data that makes our study relevant.

In [324]:
del data['name'] 
del data['description'] 
del data['city'] 

# Spoken languages

In [325]:
available_languages = {}
languages = np.array(data.languages)
for profile_languages in languages:
    profile_languages = ast.literal_eval(profile_languages)
    if len(profile_languages)>0:
        for language in profile_languages:
            if language not in available_languages:
                available_languages[language] = 1
            else:
                available_languages[language] += 1

In [326]:
len(available_languages)

102

In [327]:
lang = np.array(list(available_languages.keys()))
count_lang = list(available_languages.values())
sorted_count_lang = np.argsort(count_lang)[-10:]
most_spoken_languages = lang[sorted_count_lang]

In [328]:
most_spoken_languages

array(['Portuguese', 'Russian', 'Arabic', 'Urdu', 'Italian', 'Hindi',
       'German', 'Spanish', 'French', 'English'], dtype='<U13')

In [329]:
n = len(data)
english, french, spanish, arabic, german, italian, hindi, urdu, russian, portuguese = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)

for i, lang in enumerate(languages):
    lang=ast.literal_eval(lang)
    if 'English' in lang:
        english[i]=1
    if 'French' in lang:
        french[i]=1
    if 'Spanish' in lang:
        spanish[i]=1
    """
    if 'Arabic' in lang:
        arabic[i]=1
    if 'German' in lang:
        german[i]=1
    if 'Italian' in lang:
        italian[i]=1
    if 'Hindi' in lang:
        hindi[i]=1
    if 'Urdu' in lang:
        urdu[i]=1
    if 'Russian' in lang:
        russian[i]=1
    if 'Portuguese' in lang:
        portuguese[i]=1
    """

In [330]:
del data['languages']

data['lang_english'] = english
data['lang_french'] = french
data['lang_spanish'] = spanish

"""
data['arabic'] = arabic
data['german'] = german
data['italian'] = italian
data['hindi'] = hindi
data['urdu'] = urdu
data['russian'] = russian
data['portuguese'] = portuguese
"""

"\ndata['arabic'] = arabic\ndata['german'] = german\ndata['italian'] = italian\ndata['hindi'] = hindi\ndata['urdu'] = urdu\ndata['russian'] = russian\ndata['portuguese'] = portuguese\n"

# Industry expertises

In [331]:
available_expertise = {}
industry_expertise = np.array(data.industry_expertise)
for profile_expertise in industry_expertise:
    profile_expertise = ast.literal_eval(profile_expertise)
    if len(profile_expertise)>0:
        for expertise in profile_expertise:
            if expertise not in available_expertise:
                available_expertise[expertise] = 1
            else:
                available_expertise[expertise] += 1

In [332]:
len(available_expertise)

65

In [333]:
exper = np.array(list(available_expertise.keys()))
count_exper = list(available_expertise.values())
sorted_count_exper = np.argsort(count_exper)[-10:]
most_imp_exper = exper[sorted_count_exper]

In [334]:
most_imp_exper

array(['Publishing', 'Healthcare', 'Travel', 'Internet', 'Technology',
       'IT', 'Media', 'Advertising', 'Education', 'Marketing'],
      dtype='<U18')

In [335]:
n = len(data)
marketing, education, advertising, media, it, internet, technology, entertainment, travel, art = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)

for i, exper in enumerate(industry_expertise):
    exper=ast.literal_eval(exper)
    if 'Marketing' in exper:
        marketing[i]=1
    if 'Education' in exper:
        education[i]=1
    if 'Advertising' in exper:
        advertising[i]=1
"""
    if 'Media' in exper:
        media[i]=1
    if 'IT' in exper:
        it[i]=1
    if 'Internet' in exper:
        internet[i]=1
    if 'Technology' in exper:
        technology[i]=1
    if 'Entertainment' in exper:
        entertainment[i]=1
    if 'Travel' in exper:
        travel[i]=1
    if 'Art' in exper:
        art[i]=1
"""

"\n    if 'Media' in exper:\n        media[i]=1\n    if 'IT' in exper:\n        it[i]=1\n    if 'Internet' in exper:\n        internet[i]=1\n    if 'Technology' in exper:\n        technology[i]=1\n    if 'Entertainment' in exper:\n        entertainment[i]=1\n    if 'Travel' in exper:\n        travel[i]=1\n    if 'Art' in exper:\n        art[i]=1\n"

In [336]:
del data['industry_expertise']

data['exper_education'] = education
data['exper_advertising'] = advertising
data['exper_marketing'] = marketing

"""
data['exper_media'] = media
data['exper_it'] = it
data['exper_internet'] = internet
data['exper_technology'] = technology
data['exper_entertainment'] = entertainment
data['exper_travel'] = travel
data['exper_art'] = art
"""

"\ndata['exper_media'] = media\ndata['exper_it'] = it\ndata['exper_internet'] = internet\ndata['exper_technology'] = technology\ndata['exper_entertainment'] = entertainment\ndata['exper_travel'] = travel\ndata['exper_art'] = art\n"

# Recently active workers

In [337]:
data.last_active = pd.to_datetime(data.last_active)
threshold = datetime.datetime.strptime('20-03-2021', '%d-%m-%Y') 
# Choose date to qualify if worker was recently active

data.last_active = threshold < data.last_active
data.last_active = data.last_active*1

# Countries of work

In [338]:
location = data['country']

country = [ l.split(',')[-1].replace(" ", "") for l in location ]
count = Counter(country).most_common(10)

print(count)

[('UnitedKingdom', 3178), ('India', 511), ('Pakistan', 397), ('Kenya', 185), ('France', 150), ('Germany', 136), ('Spain', 136), ('Bangladesh', 132), ('Italy', 126), ('Nigeria', 118)]


In [339]:
values = data['country']
counts = pd.value_counts(values)
mask = values.isin(counts.index)
coun = pd.get_dummies(values[mask])
del data['country']
data['country_UnitedKingdom'] = coun[' United Kingdom']
data['country_India'] = coun[' India']
data['country_Pakistan'] = coun[' Pakistan']
data['country_Kenya'] = coun[' Kenya']

# Skills

In [340]:
available_skill = {}
skills = np.array(data.skills)
for profile_skill in skills:
    profile_skill = ast.literal_eval(profile_skill)
    if len(profile_skill)>0:
        for skill in profile_skill:
            if skill not in available_skill:
                available_skill[skill] = 1
            else:
                available_skill[skill] += 1

In [341]:
len(available_skill)

2148

In [342]:
sk = np.array(list(available_skill.keys()))
count_sk = list(available_skill.values())
sorted_count_sk = np.argsort(count_sk)[-10:]
most_imp_sk = sk[sorted_count_sk]

In [343]:
most_imp_sk

array(['web writing', 'social media marketing', 'article',
       'search engine optimization', 'editing', 'creative writing',
       'copywriting', 'proofreading', 'blog writing', 'content writing'],
      dtype='<U51')

In [344]:
n = len(data)
creative_writing, copywriting, proofreading, blog_writing, content_writing = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)

for i, sk in enumerate(skills):
    sk=ast.literal_eval(sk)
    if 'creative writing' in sk:
        creative_writing[i]=1
    if 'copywriting' in sk:
        copywriting[i]=1
    if 'proofreading' in sk:
        proofreading[i]=1
    if 'blog writing' in sk:
        blog_writing[i]=1
    if 'content writing' in sk:
        content_writing[i]=1

In [345]:
del data['skills']

data['skill_creative_writing'] = creative_writing
data['skill_Copywriting'] = copywriting
data['skill_proofreading'] = proofreading
data['skill_blog_writing'] = blog_writing
data['skill_content_writing'] = content_writing

# Gender

In [346]:
values = data['gender']
counts = pd.value_counts(values)
mask = values.isin(counts[0:3].index)
gen = pd.get_dummies(values[mask])

del data['gender']
data = pd.concat([data, gen], axis=1)

# Final data

In [347]:
data.head(5)

,rating,nb_reviews,Price/hour in USD,nb_projects,nb_buyers,last_active,score,lang_english,lang_french,lang_spanish,...,country_Pakistan,country_Kenya,skill_creative_writing,skill_Copywriting,skill_proofreading,skill_blog_writing,skill_content_writing,female,male,unknown
0,4.9,4694,34.74,7672,4168,1,10.043,1.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,1.0,1,0,0
1,5.0,1190,138.95,1085,533,1,8.691,1.0,0.0,0.0,...,0,0,0.0,0.0,0.0,1.0,0.0,0,0,1
3,4.9,598,20.00,721,376,1,7.983,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0,1
4,4.9,135,27.79,140,100,1,6.496,1.0,1.0,1.0,...,0,0,0.0,0.0,1.0,0.0,0.0,0,0,1
5,4.9,554,22.23,404,293,1,7.907,1.0,1.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [348]:
print(data.isna().values.any())
print(data.isna().sum().sum())
print(data.isna().sum())

False
0
rating                    0
nb_reviews                0
Price/hour in USD         0
nb_projects               0
nb_buyers                 0
last_active               0
score                     0
lang_english              0
lang_french               0
lang_spanish              0
exper_education           0
exper_advertising         0
exper_marketing           0
country_UnitedKingdom     0
country_India             0
country_Pakistan          0
country_Kenya             0
skill_creative_writing    0
skill_Copywriting         0
skill_proofreading        0
skill_blog_writing        0
skill_content_writing     0
female                    0
male                      0
unknown                   0
dtype: int64


In [349]:
data.describe()

,rating,nb_reviews,Price/hour in USD,nb_projects,nb_buyers,last_active,score,lang_english,lang_french,lang_spanish,...,country_Pakistan,country_Kenya,skill_creative_writing,skill_Copywriting,skill_proofreading,skill_blog_writing,skill_content_writing,female,male,unknown
count,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,...,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000,7073.000000
mean,4.834398,43.217447,30.884676,46.545879,30.039446,0.640888,3.881086,0.792309,0.103634,0.092464,...,0.056129,0.026156,0.216740,0.287431,0.334653,0.336067,0.438993,0.481832,0.353174,0.164994
std,0.667073,185.349740,34.608079,274.213685,166.041998,0.479774,1.568235,0.405683,0.304806,0.289701,...,0.230187,0.159610,0.412053,0.452596,0.471902,0.472395,0.496299,0.499705,0.477990,0.371201
min,0.000000,0.000000,8.340000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.900000,3.000000,14.500000,3.000000,2.000000,0.000000,2.773000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,8.000000,24.170000,8.000000,6.000000,1.000000,3.714000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,27.000000,35.000000,26.000000,19.000000,1.000000,4.913000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,5.000000,8647.000000,1354.740000,16306.000000,10343.000000,1.000000,10.634000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [350]:
X = data.drop(['Price/hour in USD'], axis=1)
y = data['Price/hour in USD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

model = sm.OLS(y_train.astype(float), X_train.astype(float))
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:      Price/hour in USD   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     16.49
Date:                Mon, 17 May 2021   Prob (F-statistic):           1.11e-63
Time:                        11:08:56   Log-Likelihood:                -26364.
No. Observations:                5304   AIC:                         5.278e+04
Df Residuals:                    5280   BIC:                         5.293e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
rating                     0